In [1]:
# %matplotlib widget
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import skimage.data
import numpy as np
import cupy as cp

from src.pyvsnr import vsnr2d
from src.pyvsnr.utils import curtains_addition, stripes_addition, add_gaussian_noise
from utils import measure_vsnr_cuda, measure_vsnr_numpy, measure_vsnr_cupy, print_psnr, plot_results, print_max_diff, peak_signal_noise_ratio
xp = cp

### Image preparation

In [ ]:
img = skimage.data.camera()
img = img / 255  # normalize to [0,1]

filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
maxit = 20

In [ ]:
# Add vertical stripes (Dirac filter)
noisy_img = stripes_addition(img, amplitude=0.2)

### CuPy vs CUDA vs NumPy Speed Comparison

In [ ]:
%timeit measure_vsnr_cuda(noisy_img, filters, nite=maxit)
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=maxit)

In [ ]:
if xp==cp:
    %timeit measure_vsnr_cupy(noisy_img, filters, maxit=maxit)
    img_corr_py = vsnr2d(noisy_img, filters, maxit=maxit, algo='cupy')
else:
    %timeit measure_vsnr_numpy(noisy_img, filters, maxit=maxit)
    img_corr_py = vsnr2d(noisy_img, filters, maxit=maxit, algo='numpy')

### PSNR

In [ ]:
print_max_diff(img_corr_py, img_corr_cuda, xp) # sometimes precision might be >1e-6
print_psnr(img, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img, noisy_img, img_corr_py, img_corr_cuda, xp, save_plots=True, title='camera_stripes.png')

### Test Gaussian noise

In [ ]:
img=skimage.data.camera()/255

# Add gaussian noise
noisy_img = add_gaussian_noise(img)


filters=[{'name':'Dirac', 'noise_level':0.35}]
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=20)

if xp==cp:
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, maxit=20)
else:
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, maxit=20)

print_max_diff(img_corr_cuda, img_corr_py, xp)
print_psnr(img, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img, noisy_img, img_corr_py, img_corr_cuda, xp, save_plots=True, title='camera_gaussian.png')

### Test Curtains

In [ ]:
# Generate image with noise
img_base=skimage.data.camera()
img_base=img_base/255
noisy_img = curtains_addition(img_base, amplitude=0.2, angle=50)

# Process image
filters = [{'name':'Gabor', 'noise_level':20, 'sigma':(3,40), 'theta':50}]
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=maxit)
if xp==cp:
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, maxit=maxit)
else:
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, maxit=maxit)

print_max_diff(img_corr_cuda, img_corr_py, xp)
print_psnr(img_base, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img_base, noisy_img, img_corr_py, img_corr_cuda, xp, save_plots=True, title='camera_curtains.png')

### Test FIBSEM

In [ ]:
#load image fib_sem.tif
img = skimage.io.imread('./images/fib_sem.tif')/255

filters = [{'name':'Gabor', 'noise_level':30, 'sigma':(1,30), 'theta':358}]

# process image
img_corr_cuda = measure_vsnr_cuda(img, filters, nite=maxit)

if xp==cp:
    img_corr_py = measure_vsnr_cupy(img, filters, maxit=maxit)
else:
    img_corr_py = measure_vsnr_numpy(img, filters, maxit=maxit)

# print PSNR & save plots
print_max_diff(img_corr_cuda, img_corr_py, xp) # sometimes precision might be >1e-6
print_psnr(img, img, img_corr_py, img_corr_cuda)

plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.imshow(img, cmap='gray')
plt.title('Original')

plt.subplot(1,2,2)
plt.imshow(img_corr_py, cmap='gray')
plt.title('Corrected')

plt.savefig('./images/fib_sem_corr.png', bbox_inches='tight')


### Minimal example

In [ ]:
# Generate image with noise
img=0.5*np.ones((512,512), dtype=np.float32)
noisy_img = stripes_addition(img, amplitude=0.2, norm=False)


# Process image
filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=maxit, norm=False)
if xp==cp:
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, maxit=maxit, norm=False)
else:
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, maxit=maxit, norm=False)

print_max_diff(img_corr_cuda, img_corr_py, xp)
print_psnr(img, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img, noisy_img, img_corr_py, img_corr_cuda, xp, vmin=0.3, vmax=0.7)

### Minimal example 2

In [ ]:
# Generate image with noise
img=0.5*np.ones((512,512), dtype=np.float32)
noisy_img = curtains_addition(img, amplitude=0.6, angle=50, norm=False)


# Process image
filters = [{'name':'Gabor', 'noise_level':60, 'sigma':(3,40), 'theta':50}]
img_corr_cuda = measure_vsnr_cuda(noisy_img, filters, nite=maxit, norm=False)
if xp==cp:
    img_corr_py = measure_vsnr_cupy(noisy_img, filters, maxit=maxit, norm=False)
else:
    img_corr_py = measure_vsnr_numpy(noisy_img, filters, maxit=maxit, norm=False)

print(f"Mean of original image:         {img.mean()}")
print(f"Mean of noisy image:            {noisy_img.mean()}")
print(f"Mean of corrected image (CuPy): {img_corr_py.mean()}")
print(f"Mean of corrected image (CUDA): {img_corr_cuda.mean()}")

print_max_diff(img_corr_cuda, img_corr_py, xp)
print_psnr(img, noisy_img, img_corr_py, img_corr_cuda)
plot_results(img, noisy_img, img_corr_py, img_corr_cuda, xp, vmin=0.3, vmax=0.7)

### PSNR Analysis

In [ ]:
# Generate image with noise
img=skimage.data.camera()
img=img/255
noisy_img = stripes_addition(img, amplitude=0.2)

# Process image
filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
maxit = 50
psnr_cuda = []
for i in range(1,maxit):
    img_clean = vsnr2d(noisy_img, filters, maxit=i, algo='cupy', verbose=True)
    psnr_cuda.append(peak_signal_noise_ratio(img, img_clean))

    #Plotting each iteration
    # if i%4==0:
    #     plt.figure(figsize=(5, 5))
    #     plt.imshow(img_clean.get())
    #     plt.title(f"Denoised Image w/ maxit={i}")
    #     plt.tight_layout()

# Determine optimal number of iterations
maxit_optimal = np.argmax(psnr_cuda) + 1

print('Optimal number of iterations: ', maxit_optimal)
# Process image with optimal number of iterations
img_clean_optimal = measure_vsnr_cupy(noisy_img, filters, maxit=maxit_optimal)

if type(img_clean_optimal) == cp.ndarray:
    img_clean_optimal = img_clean_optimal.get()

# Plot PSNR graph
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(psnr_cuda)
plt.xlabel('N Iteration')
plt.ylabel('PSNR')

# Plot original image
plt.subplot(1, 3, 2)
plt.imshow(noisy_img)
plt.title('Noisy Image')

# Plot denoised image
plt.subplot(1, 3, 3)
plt.imshow(img_clean_optimal)
plt.title(f"Denoised Image w/ optimal maxit={maxit_optimal}")

plt.tight_layout()

### CVG Criteria Analysis

In [ ]:
img=0.5*np.ones((512,512), dtype=np.float32)
noisy_img = stripes_addition(img, amplitude=0.2, norm=False)
filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
maxit = 50

img_corr, cvg_criterias = vsnr2d(noisy_img, filters, maxit=maxit, norm=False, return_cvg=True)

print(cvg_criterias)
# plot cvg criteria
plt.figure(figsize=(15, 5))
plt.semilogy(cvg_criterias)
plt.xlabel('N Iteration')
plt.ylabel('Cvg Criteria')

In [ ]:
# Cvg criteria analysis on camera.tif image
img= skimage.data.camera()
noisy_img = stripes_addition(img, amplitude=0.2)

# Process image
filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
maxit = 50

img_corr, cvg_criterias = vsnr2d(noisy_img, filters, maxit=maxit, algo='cupy' ,cvg_threshold=1e-2, return_cvg=True)

# plot cvg criteria
plt.figure(figsize=(15, 5))
plt.semilogy(cvg_criterias)
plt.xlabel('N Iteration')
plt.ylabel('Cvg Criteria')

In [ ]:
# cvg criteria analysis on fib_sem.tif
img= skimage.io.imread('./images/fib_sem.tif')/255
filters = [{'name':'Gabor', 'noise_level':30, 'sigma':(1,30), 'theta':358}]
maxit = 50

img_corr, cvg_criterias = vsnr2d(img, filters, maxit=maxit, return_cvg=True)

# plot cvg criteria
plt.figure(figsize=(15, 5))
plt.semilogy(cvg_criterias)
plt.xlabel('N Iteration')
plt.ylabel('Cvg Criteria')


In [ ]:
# cvg criteria analysis on camera with curtains
img= skimage.data.camera()/255
noisy_img = curtains_addition(img, amplitude=0.2, angle=50)

# Process image
filters = [{'name':'Gabor', 'noise_level':20, 'sigma':(3,40), 'theta':50}]
maxit = 50

img_corr, cvg_criterias = vsnr2d(noisy_img, filters, maxit=maxit, algo='auto', return_cvg=True)

# plot cvg criteria
plt.figure(figsize=(15, 5))
plt.semilogy(cvg_criterias)
plt.xlabel('N Iteration')
plt.ylabel('Cvg Criteria')


### Performance Analysis

In [ ]:
# performance comparison between numpy and cupy and cuda based on the number of iterations
import time

img=skimage.data.camera()
img=img/255
noisy_img = stripes_addition(img, amplitude=0.2)

# Process image
filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
maxit = 50
time_cuda = []
time_cupy = []
time_numpy = []

# CUDA
for i in range(1,maxit):
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, maxit=i, algo='cuda')
    end = time.perf_counter()
    time_cuda.append(end-start)

# CuPy
for i in range(1,maxit):
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, maxit=i, algo='cupy')
    end = time.perf_counter()
    time_cupy.append(end-start)

# NumPy
for i in range(1,maxit):
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, maxit=i, algo='numpy')
    end = time.perf_counter()
    time_numpy.append(end-start)

# Plot time taken on one graph
plt.figure(figsize=(15, 5))
plt.plot(time_numpy, label='NumPy')
plt.plot(time_cupy, label='CuPy')
plt.plot(time_cuda, label='CUDA')
plt.xlabel('N Iterations')
plt.ylabel('time taken (s)')
plt.legend()


In [ ]:
# performance comparison between numpy and cupy and cuda based on the size of the image
sizes = [256, 512, 1024, 2048]
    
time_cuda = []
time_cupy = []
time_numpy = []

for size in sizes:
    # Generate image with noise
    img=0.5*np.ones((size,size), dtype=np.float32)
    noisy_img = stripes_addition(img, amplitude=0.2)

    # Process image
    filters = [{'name':'Gabor', 'noise_level':100, 'sigma':(1000,0.1), 'theta':0}]
    maxit = 10

    # CUDA
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, maxit=maxit, algo='cuda')
    end = time.perf_counter()
    time_cuda.append(end-start)

    # CuPy
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, maxit=maxit, algo='cupy')
    end = time.perf_counter()
    time_cupy.append(end-start)

    # NumPy
    start = time.perf_counter()
    vsnr2d(noisy_img, filters, maxit=maxit, algo='numpy')
    end = time.perf_counter()
    time_numpy.append(end-start)

# Plot time taken on one graph
plt.figure(figsize=(15, 5))
plt.plot(sizes, time_numpy, label='NumPy')
plt.plot(sizes, time_cupy, label='CuPy')
plt.plot(sizes, time_cuda, label='CUDA')
plt.xlabel('Image Size (px)')
plt.ylabel('time (s)')
plt.legend()